### A Model intended to generate text the same likeness as the input text file; hence the 'echo' in 'echoGPT'


In [12]:
import torch

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-07 01:49:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-03-07 01:49:01 (78.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

In [11]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hello world!$"))
print(decode(encode("hello world!$")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2, 3]
hello world!$


In [14]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:1000])

torch.Size([1115394]) torch.int64


In [15]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [28]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [36]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [38]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4522743225097656


In [39]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


DUKIVisun Casshe wisthiot s.
LUK:

NGOLI io e alllker s?j$NCowens l het hislaspicobar, heay ind, cigigeluandac! thaforo nont
SLO:
Ange ive nn I ou m,
UCENTheanp'Lbet bazzl
TEEXNore t b'Thathon:
sous min'd ne st wousis s lingilo whee,
K:
Toow'e's,
D:
NGLEng, do te! ase may sin ceecate.
God? d
Aw ht hongur vet ouley, tonches s ousour mes o be angaxHatiminthafe atnt ur whis heand ay out sitourdy spehacugnting akefo tofed werore mo d htreldeg ise se inon ime weagsone the kener bs s oweclishentens t 


In [61]:
a = torch.tril(torch.ones(3, 3))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [62]:
torch.sum(a, 1, keepdim=True)

tensor([[1.],
        [2.],
        [3.]])

In [63]:
a = a / torch.sum(a, 1, keepdim=True)
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [64]:
b = torch.randint(0,10,(3,2)).float()
b

tensor([[7., 6.],
        [9., 6.],
        [3., 1.]])

[1.0000, 0.0000, 0.0000] *
                          [7]
                          [9]
                          [3] = [1 * 7 + 0 * 9 + 0 * 3] = [7]

[1.0000, 0.0000, 0.0000] *
                      [6]
                      [6]
                      [1] = [1 * 6 + 0 * 6 + 0 * 1] = [6]

[0.5000, 0.5000, 0.0000]* [7]
                          [9]
                          [3] = [0.5 * 7 + 0.5 * 9 + 0 * 3] = [8]

[0.5000, 0.5000, 0.0000] *
                      [6]
                      [6]
                      [3] = [0.5 * 6 + 0.5 * 6 + 0 * 3] = [3+3+0] = [6]


In [65]:
c = a @ b
c

tensor([[7.0000, 6.0000],
        [8.0000, 6.0000],
        [6.3333, 4.3333]])

In [66]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [76]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [98]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [104]:
# [ 0.1808, -0.0700],
# [-0.3596, -0.9152],
# whats being done in the for loop cell below (manual here)
print(((0.1808 + -0.3596 )) / 2)
print(((-0.0700 + -0.9152 )) / 2)

-0.0894
-0.49260000000000004
-0.291
-0.5820000000000001


In [105]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [107]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [79]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [80]:
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [108]:
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

Self attention via softmax normalization

In [85]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [86]:
wei = torch.zeros((T,T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [87]:
wei = wei.masked_fill(tril == 0, float('-inf'))

In [88]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [90]:
xbow3 = wei @ x
xbow3

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [115]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
print('B =', B)
print('T =', T)
print('C =', C)

B = 4
T = 8
C = 32


In [116]:
x = torch.randn(B,T,C)
x[:2, :, :5]

tensor([[[ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258],
         [-0.6631, -0.2513,  1.0101,  0.1215,  0.1584],
         [-0.8345,  0.5978, -0.0514, -0.0646, -0.4970],
         [-1.6669, -1.3651, -0.1655,  0.9623,  0.0315],
         [-2.0555,  1.8275,  1.3035, -0.4501,  1.3471],
         [-0.8961,  0.0662, -0.0563,  2.3412, -2.7234],
         [ 0.1910, -0.3425,  1.7955,  1.3915,  1.0785],
         [-0.5819, -0.2208,  0.0135, -0.3057, -0.0304]],

        [[ 0.4562, -1.0917, -0.8207,  1.8634,  0.8148],
         [ 0.0210,  1.0060, -1.2492,  0.2441, -0.6387],
         [ 2.2007, -0.2195,  0.5427,  2.5867, -0.4687],
         [ 0.2922,  1.3143,  1.2607, -0.3505, -2.0660],
         [ 0.1275, -0.0560,  0.8315, -0.5512,  1.0477],
         [ 0.3091,  1.1661, -2.1821, -1.0422,  1.0207],
         [ 0.0943, -0.3156,  0.7850, -0.8699, -1.6525],
         [ 0.6455, -0.3313, -1.0390,  0.9112,  1.2984]]])

In [113]:
# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

In [117]:
k = key(x)   # (B, T, 16)
k[:2,:,:5]

tensor([[[-6.5674e-01,  2.8302e-02,  9.4470e-03, -6.9949e-01, -3.6043e-01],
         [-3.9319e-01,  8.2196e-01, -7.0274e-01,  9.5429e-02, -1.2218e-01],
         [ 2.1567e-01, -3.5065e-01,  2.1671e-03,  4.2317e-01, -2.2844e-01],
         [ 8.9999e-01, -1.2723e-01,  5.4581e-01,  4.2544e-01, -4.5128e-01],
         [ 3.3199e-02,  5.8858e-01, -4.4368e-01,  3.7748e-01, -6.8257e-01],
         [ 2.0985e-01,  4.3915e-02, -7.0198e-02,  7.2701e-02, -2.0124e-01],
         [ 6.1001e-01, -3.2841e-01, -8.5571e-01,  8.5427e-01,  7.8055e-01],
         [ 1.4591e-01,  1.3493e-01, -2.3353e-01, -4.1732e-02,  2.9277e-01]],

        [[ 1.1104e+00, -8.7192e-01,  7.0978e-01,  3.6331e-01,  2.0670e-01],
         [ 3.2561e-01,  5.7866e-01,  5.4575e-01, -7.2274e-01,  1.2343e+00],
         [-1.5634e-02, -5.4639e-01,  3.0958e-01,  3.5532e-01,  5.9885e-01],
         [-2.4959e-01,  2.7492e-01,  2.6894e-01, -3.6563e-01, -3.2585e-01],
         [ 3.9181e-01,  5.7756e-01,  1.3630e-01, -3.3129e-01,  3.4955e-01],
         [

In [133]:
q = query(x) # (B, T, 16)
q[:1, :1, :]

tensor([[[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007,
          -0.5239, -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,
           0.2862,  0.5710]]], grad_fn=<SliceBackward0>)

In [134]:
k[:1, :1 , :]

tensor([[[-0.6567,  0.0283,  0.0094, -0.6995, -0.3604,  0.8376, -0.4446,
           0.1228,  0.6276, -0.6222,  0.3483,  0.2411,  0.5409, -0.2605,
           0.3612, -0.0436]]], grad_fn=<SliceBackward0>)

In [135]:
k[:1, :1 , :].transpose(-2,-1)

tensor([[[-0.6567],
         [ 0.0283],
         [ 0.0094],
         [-0.6995],
         [-0.3604],
         [ 0.8376],
         [-0.4446],
         [ 0.1228],
         [ 0.6276],
         [-0.6222],
         [ 0.3483],
         [ 0.2411],
         [ 0.5409],
         [-0.2605],
         [ 0.3612],
         [-0.0436]]], grad_fn=<TransposeBackward0>)

In [123]:
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)
wei[:2,:,:5]

tensor([[[ 4.3221e-01, -3.4657e-01, -1.4631e+00,  3.2895e-02,  8.7217e-01],
         [-1.2225e+00,  4.2318e-02,  1.6674e+00,  4.7402e-01, -3.9212e-01],
         [-9.0737e-01, -2.5469e+00,  1.4997e+00, -6.8395e-01,  2.7916e-01],
         [-1.4066e+00,  3.1887e-01,  9.5003e-01, -4.7187e-01,  6.4279e-01],
         [-3.2678e-01, -3.8157e+00, -3.3920e-01,  7.4762e-01,  6.4912e-01],
         [ 8.7737e-01,  2.1680e+00, -2.4931e-01, -1.4231e+00,  1.4555e-01],
         [-2.3583e-02,  9.0819e-01,  8.2675e-01, -8.6153e-01,  3.3840e-02],
         [ 7.6655e-01,  1.1162e+00,  2.3495e+00, -5.3060e-01, -2.1206e+00]],

        [[-1.7818e+00,  2.5224e+00, -1.5429e+00, -1.2020e-01,  2.3735e+00],
         [ 1.0509e-01, -1.1810e+00,  8.5433e-01,  1.5353e-01, -1.5560e+00],
         [-1.1781e+00,  1.1836e+00,  1.8008e-01,  9.7110e-01,  4.9608e-02],
         [ 2.7031e-01,  7.3144e-01, -5.9977e-01,  5.1165e-01,  9.2963e-01],
         [ 5.1286e-01, -6.3709e-01, -2.2027e-01,  7.8781e-01,  6.5718e-01],
         [

In [138]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [140]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei[:1, :, :]

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3636, 0.6364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2375, 0.2224, 0.5401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2013, 0.2543, 0.3267, 0.2177, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1848, 0.1627, 0.1845, 0.2382, 0.2297, 0.0000, 0.0000, 0.0000],
         [0.1624, 0.2488, 0.1456, 0.1404, 0.1494, 0.1533, 0.0000, 0.0000],
         [0.1355, 0.1562, 0.1534, 0.1286, 0.1363, 0.1485, 0.1415, 0.0000],
         [0.1204, 0.1254, 0.1756, 0.1121, 0.1098, 0.1186, 0.1185, 0.1196]]],
       grad_fn=<SliceBackward0>)

In [142]:
v = value(x)
v[:1, :, :]

tensor([[[-0.8604, -0.6466,  0.0754, -0.6363, -0.1464,  0.3840, -0.3053,
          -0.6389,  0.0544,  0.0762,  0.2364, -0.1792, -0.2908,  0.3348,
          -0.9431, -0.1245],
         [-0.8963, -0.1909,  0.0988, -1.0337, -0.4137,  0.9316, -0.1586,
          -0.1791,  0.1190,  0.6819,  0.0265,  0.5562, -0.1615,  0.1101,
           0.0846, -0.6664],
         [-0.7785,  0.2959, -0.2016, -0.7096,  0.4515,  0.4565,  0.2554,
           0.4989,  0.0964,  0.5159,  0.3097, -0.3569, -0.2772, -0.3573,
           0.4011,  0.0285],
         [ 0.6401, -0.1027,  0.4378,  0.6083,  0.5365,  0.3228,  0.0042,
           0.6595,  0.4381, -0.3499, -1.4774, -0.4494,  0.6942, -0.0524,
           0.1576,  0.2287],
         [ 0.3436,  0.7927,  1.3172,  0.2551, -0.1853,  0.3615,  1.2092,
           0.6290,  0.1703,  0.3958,  0.2562, -0.1899, -0.8195,  0.0613,
           0.2854, -1.2319],
         [ 1.6714, -0.7109,  1.5069,  2.0694,  0.4068,  0.2825,  0.9223,
           1.3963, -1.0078,  0.0742,  0.2363,  0.828

In [145]:
out = wei @ v
print(wei.shape)
print(v.shape)
out[:1]

torch.Size([4, 8, 8])
torch.Size([4, 8, 16])


tensor([[[-0.8604, -0.6466,  0.0754, -0.6363, -0.1464,  0.3840, -0.3053,
          -0.6389,  0.0544,  0.0762,  0.2364, -0.1792, -0.2908,  0.3348,
          -0.9431, -0.1245],
         [-0.8832, -0.3566,  0.0903, -0.8892, -0.3165,  0.7325, -0.2120,
          -0.3463,  0.0955,  0.4617,  0.1028,  0.2888, -0.2085,  0.1918,
          -0.2891, -0.4693],
         [-0.8242, -0.0362, -0.0690, -0.7643,  0.1171,  0.5450,  0.0301,
           0.0779,  0.0914,  0.4484,  0.2293, -0.1116, -0.2547, -0.0890,
           0.0115, -0.1624],
         [-0.5162, -0.1044,  0.0697, -0.4904,  0.1296,  0.5337, -0.0174,
           0.1324,  0.1681,  0.2812, -0.1661, -0.1090, -0.0391, -0.0328,
          -0.0029, -0.1355],
         [-0.2171,  0.0617,  0.3997, -0.2132,  0.0742,  0.4667,  0.2437,
           0.2464,  0.1907,  0.2278, -0.1879, -0.1592, -0.1541,  0.0155,
           0.0166, -0.3547],
         [-0.0786, -0.1145,  0.4968, -0.0231,  0.0491,  0.5033,  0.2708,
           0.3250, -0.0151,  0.2785, -0.0429,  0.092